In [1]:
import os
import joblib

# تأكد أن BASE_PATH يشير للمسار الصحيح حيث توجد ملفات vectorizer
BASE_PATH = os.path.abspath(os.path.join(os.getcwd(), "..", "vectorize", "saved_models"))

def load_vectorizer(name: str, vectorizer_type: str = "tfidf"):
    path = os.path.join(BASE_PATH, vectorizer_type, f"{name}_vectorizer.joblib")
    print(f"Loading vectorizer from: {path}")
    model = joblib.load(path)
    print(f"Loaded object type: {type(model)}")
    return model

datasets = ["antique", "beir"]

for dataset_name in datasets:
    file_prefix = f"{dataset_name}_all"
    print(f"\nTesting dataset: {dataset_name}")
    try:
        vectorizer = load_vectorizer(file_prefix)
        assert "TfidfVectorizer" in str(type(vectorizer)), "Loaded object is not a TfidfVectorizer!"
        print(f"✔️ {dataset_name} vectorizer loaded correctly and is a TfidfVectorizer.")
    except Exception as e:
        print(f"❌ Error loading vectorizer for {dataset_name}: {e}")


Evaluating query 3097310
[DISK] Loading embedding model all-MiniLM-L6-v2
[DISK] Loading document embeddings for antique
[EMBEDDING] عرض أعلى الكل نتائج مرتبة:
🔹 Rank: 1
   Doc ID: 469449
   Score: 0.680869
--------------------------------------------------
🔹 Rank: 2
   Doc ID: 657104
   Score: 0.680719
--------------------------------------------------
🔹 Rank: 3
   Doc ID: 1029451
   Score: 0.667408
--------------------------------------------------
🔹 Rank: 4
   Doc ID: 658159
   Score: 0.662408
--------------------------------------------------
🔹 Rank: 5
   Doc ID: 910556
   Score: 0.630224
--------------------------------------------------
Evaluating query 3910705
[CACHE] Using cached embedding model all-MiniLM-L6-v2
[EMBEDDING] عرض أعلى الكل نتائج مرتبة:
🔹 Rank: 1
   Doc ID: 503266
   Score: 0.936756
--------------------------------------------------
🔹 Rank: 2
   Doc ID: 503239
   Score: 0.852432
--------------------------------------------------
🔹 Rank: 3
   Doc ID: 503321
   Score

In [4]:
import sys
import os

# إضافة مجلد الجذر (IR-project) لمسارات البحث
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from query_processing import process
import numpy as np

dataset = "beir"
query = "what is Iraq?"

vector, tokens = process(dataset, query)

print("Vector shape:", vector.shape)
print("Tokens after preprocessing:", tokens)
vector_array = vector.toarray()[0]
nonzero_indices = np.nonzero(vector_array)[0]

print(f"عدد القيم غير الصفرية: {len(nonzero_indices)}")
for idx in nonzero_indices:
    print(f"Index: {idx}, TF-IDF: {vector_array[idx]}")


Loading vectorizer from: c:\Users\HP\IR-project\vectorize\saved_models\tfidf\beir_all_vectorizer.joblib
Loaded object type: <class 'sklearn.feature_extraction.text.TfidfVectorizer'>
Vector shape: (1, 711463)
Tokens after preprocessing: ['iraq', 'iraq']
عدد القيم غير الصفرية: 1
Index: 313706, TF-IDF: 1.0


In [10]:
from ranking import match_and_rank
from evaluation import compute_map

query = "what is oppon?"

dataset = "beir"
qrels_file = f"dataBases/{dataset}_qrels.tsv"

results = match_and_rank(query, dataset)
map_score = compute_map(results, qrels_file)

print("Top ranked documents:", list(results.items())[:5])
print(f"MAP Score: {map_score:.4f}")


[1] بدء معالجة الاستعلام...
Loading vectorizer from: c:\Users\Barakat\Lectures\IR\vectorize\saved_models\tfidf\beir_all_vectorizer.joblib
Loaded object type: <class 'sklearn.feature_extraction.text.TfidfVectorizer'>
[2] تم استخراج التوكنز من الاستعلام: ['oppon']
[3] تحميل تمثيلات TF-IDF...
[4] شكل مصفوفة TF-IDF: (382545, 711463)
[5] تحميل قائمة معرفات الوثائق...
[6] عينة من doc_ids: [527629, 527630, 527631, 527632, 527633] (كلها من نوع: <class 'int'>)
[7] تحميل الفهرس المعكوس...
[*] محاولة تحميل الفهرس من: C:\Users\HP\IR-project\indexing\saved_models\inverted_index\beir_inverted_index.joblib


FileNotFoundError: لم يتم العثور على الفهرس المعكوس: C:\Users\HP\IR-project\indexing\saved_models\inverted_index\beir_inverted_index.joblib

In [5]:
from ranking import match_and_rank

query = "What is also?"
dataset = "beir"

results = match_and_rank(query, dataset)
print("Top ranked docs:", list(results.items())[:5])


[1] بدء معالجة الاستعلام...
Loading vectorizer from: c:\Users\Barakat\Lectures\IR\vectorize\saved_models\tfidf\beir_all_vectorizer.joblib
Loaded object type: <class 'sklearn.feature_extraction.text.TfidfVectorizer'>


FileNotFoundError: Document IDs file not found at: c:\Users\Barakat\Lectures\IR\vectorize\saved_models\tfidf\beir_all_doc_ids.joblib

In [14]:
import sys
import os

# إضافة مجلد الجذر (IR-project) لمسارات البحث
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)


from query_processing import process

query_vec, tokens = process("antique", "Why should we use abstinence in schools?")

Loading vectorizer from: c:\Users\Barakat\Lectures\IR\vectorize\saved_models\tfidf\antique_all_vectorizer.joblib
Loaded object type: <class 'sklearn.feature_extraction.text.TfidfVectorizer'>


In [ ]:
import sys
import os

# إضافة مجلد الجذر (IR-project) لمسارات البحث
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from storage.vector_storage import load_doc_ids

# تحميل doc_ids
doc_ids = load_doc_ids("beir_all")

# طباعة أول 10 عناصر مع نوع كل عنصر
for i, doc_id in enumerate(doc_ids[:10], 1):
    print(f"{i}: {doc_id} (type: {type(doc_id)})")


1: 527629 (type: <class 'int'>)
2: 527630 (type: <class 'int'>)
3: 527631 (type: <class 'int'>)
4: 527632 (type: <class 'int'>)
5: 527633 (type: <class 'int'>)
6: 527634 (type: <class 'int'>)
7: 527635 (type: <class 'int'>)
8: 527636 (type: <class 'int'>)
9: 527637 (type: <class 'int'>)
10: 527638 (type: <class 'int'>)


In [13]:
import sys
import os

# إضافة مجلد الجذر (IR-project) لمسارات البحث
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)
from storage.vector_storage import load_doc_ids

# استدعاء التابع مع dataset name
doc_ids = load_doc_ids("beir_all")

# طباعة أول 5 عناصر مع نوع كل عنصر
for i, doc_id in enumerate(doc_ids[:5], 1):
    print(f"{i}: {doc_id} (type: {type(doc_id)})")


1: 527629 (type: <class 'int'>)
2: 527630 (type: <class 'int'>)
3: 527631 (type: <class 'int'>)
4: 527632 (type: <class 'int'>)
5: 527633 (type: <class 'int'>)


In [12]:
import os
import sys
import joblib

# إعداد المسار الجذري للمشروع
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from indexing.inverted_index_loader import load_inverted_index
index = load_inverted_index("beir")
sample_token = next(iter(index))
print(f"عينة من ids في الفهرس: {index[sample_token][:5]}")
print(f"نوع أول id في الفهرس: {type(index[sample_token][0])}")
from evaluation import run_evaluation
run_evaluation("antique", method="embedding")


[*] محاولة تحميل الفهرس من: C:\Users\HP\IR-project\indexing\saved_models\inverted_index\beir_inverted_index.joblib


FileNotFoundError: لم يتم العثور على الفهرس المعكوس: C:\Users\HP\IR-project\indexing\saved_models\inverted_index\beir_inverted_index.joblib